In [ ]:
PGHOST="localhost"
PGDATABASE="aact"
PGUSER="postgres"
PGPASSWORD="postgres"


In [ ]:
import psycopg2
import sys, os
import numpy as np
import pandas as pd
import pandas.io.sql as psql
import pickle as pkl

In [ ]:
from country_named_entity_recognition import find_countries

In [ ]:
conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
+" password="+ PGPASSWORD

In [ ]:
conn=psycopg2.connect(conn_string)


In [ ]:
cursor = conn.cursor()


In [ ]:
def load_data(schema, table):

    
    sql_command = "SELECT * FROM {};".format(str(table))
    print (sql_command)

    # Load the data
    
    data = pd.read_sql(sql_command, conn)

    print(data.shape)
    return (data)

In [ ]:
import os, sys, re, pandas as pd, numpy as np

In [ ]:
pd_country = pd.read_sql("select nct_id, name from ctgov.countries where name is not null", conn)

In [ ]:
def convert_to_alpha_2(text):
    matches = find_countries(text, is_georgia_probably_the_country=True)
    if len(matches) == 1:
        return matches[0][0].alpha_2
    elif len(matches) > 1:
        return "multiple"
    else:
        return "none"

In [ ]:
pd_country["country_alpha_2"] = pd_country.name.apply(convert_to_alpha_2)

In [ ]:
pd_country[pd_country["country_alpha_2"] == "multiple"].name.value_counts()

In [ ]:
pd_country[pd_country["country_alpha_2"] == "none"].name.value_counts()

In [ ]:
pd_country = pd_country.drop(columns=["name"])

In [ ]:
pd_country = pd_country.groupby("nct_id").agg(lambda x : ",".join(x))

In [ ]:
pd_country

In [ ]:
pd_country.country_alpha_2.value_counts()

In [ ]:
pd_country.rename(columns={"country_alpha_2":"country"}, inplace=True)

In [ ]:
pd_country.to_csv("annotations/country.csv")